In [4]:

import psycopg2 as db
import csv

In [5]:
#Configuração de inicialização
class Config:
    def __init__(self):
        self.config = {
            "postgres": {
                "user": "postgres",
                "password": "1234",
                "host": "127.0.0.1",
                "port": "5432",
                "database": "postgres"
            }
        }

In [6]:
class Connection(Config):
    def __init__(self):
        super().__init__()
        try:
            self.conn = db.connect(**self.config["postgres"])
            self.cur = self.conn.cursor()
        except Exception as e:
            print("Erro na conexão", e)
            exit(1)

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.commit()
        self.connection.close()

    @property
    def connection(self):
        return self.conn

    @property
    def cursor(self):
        return self.cur

    def commit(self):
        self.connection.commit()

    # fetchall retornará em determinados registros que será passado
    # instrução SQL
    def fetchall(self):
        return self.cursor.fetchall()

    # Método receberá a instrução SQL passará parâmetros ou tuplas vazias
    def execute(self, sql, params=None):
        self.cursor.execute(sql, params or ())

    # Cria uma instrução por meio de uma consulta(query)
    def query(self, sql, params=None):
        self.cursor.execute(sql, params or ())
        return self.fetchall()

In [7]:
class Pessoa(Connection):
    def __init__(self):
        # Chama o construtor da classe base usando super()
        super().__init__()
    # métodos de manipulação de registros

    def insert(self, *args):
        try:
            sql = "INSERT INTO pessoa (nome) VALUES (%s)"
            # Usa o método execute da classe base
            self.execute(sql, args)
            self.commit()
        except Exception as e:
            print("Erro ao inserir", e)

    # método inserir_csv

    def insert_csv(self, filename):
        try:
            data = csv.DictReader(open(filename, encoding="utf-8"))
            for row in data:
                self.insert(row["nome"])
                print("Registro Inserido")
        except Exception as e:
            print("Erro ao inserir csv", e)
    def delete(self, id):
        try:
            sql_s = f"SELECT * FROM pessoa WHERE id = {id}"
            if not self.query(sql_s):
                return "Registro não encontrado para deletar"
            sql_d = f"DELETE FROM pessoa WHERE id = {id}"
            self.execute(sql_d)
            self.commit()
            return "Registro deletado"
        except Exception as e:
            print("Erro ao deletar", e)
    def update(self, id, *args):
        try:
            sql = f"UPDATE pessoa SET nome = %s WHERE id = {id}"
            self.execute(sql, args)
            self.commit()
            print("Registro atualizado")
        except Exception as e:
            print("Erro ao atualizar", e)

In [8]:
if __name__ == "__main__":
    pessoa = Pessoa()
    pessoa.insert("Maria")
    pessoa.delete(18)
    pessoa.update(23, "Maria Antônio")
    pessoa.insert_csv("data.csv")

Registro atualizado


Registro atualizado
Registro Inserido
Registro Inserido
Registro Inserido
Registro Inserido
Registro Inserido
Registro Inserido
